In [10]:
#imports
import os
import gc
import pandas as pd
import numpy as np
import datetime as dt 

import matplotlib.pyplot as plt
import matplotlib.cm as cm 
import seaborn as sns

import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline 

from colorama import Fore, Style, init 
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [16]:
def PrintColor(text: str, color=Fore.BLUE, style=Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string"
    print(style + color + text + Style.RESET_ALL)